<a href="https://colab.research.google.com/github/chaezz/recomend-library/blob/master/Matrix_Factorization(200401.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df_ratings = pd.read_csv('/content/drive/My Drive/Matrix Factorization(2020)/ratings.csv')
df_movies = pd.read_csv('/content/drive/My Drive/Matrix Factorization(2020)/movies.csv')


In [0]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [0]:
df_rentlist = pd.read_pickle('/content/drive/My Drive/Test 500x500 /500x500rentlist(수정).pkl')
df_rentlist['value'] = 1
df_rentlist.head()

,회원번호,ISBN,value
0,47444.0,9788930206723,1
1,47444.0,9788991642638,1
2,47444.0,9788930232425,1
3,47444.0,9788937841989,1
4,47444.0,9788943309282,1


In [0]:
df_books = pd.read_csv('/content/drive/My Drive/Test 500x500 /itemtable500.csv', encoding='cp949')
df_books.head(10)

,Unnamed: 0,ISBN,도서명,저자,주제분류번호
0,0,9780060741693,Biscuit's day at the farm,story by Alyssa Satin Capucilli ;pictures by P...,NaN
1,1,9780198480648,Reds and blues,[story written by] Roderick Hunt ;[illustratio...,NaN
2,2,9780439113212,I am planet earth,by Jean Marzollo ;illustrated by Judith Moffatt,NaN
3,3,9780679864417,Cat traps,by Molly Coxe,NaN
4,4,9780679884491,(The) bald bandit,by Ron Roy;illustrated by John Steven Gurney,NaN
5,5,9780743461511,"Click, clack, moo",by Doreen Cronin ;pictures by Betsy Lewin,NaN
6,6,9780746054116,(The amazing adventures of)Ulysses,"retold by Vivian Webb,Heather Amery;adapted by...",NaN
7,7,9780746054925,Animal legends,retold by Carol Watson;illustrated by Nick Price,NaN
8,8,9781405052306,(The) Gruffalo' s Child,by Julia Donaldson;illustrated by Axel Scheffler,NaN
9,9,9781409505488,Alice through the looking-glass,by Lewis Carroll;adapted by Lesley Sims;illust...,NaN


In [0]:
df_user_book_ratings = df_rentlist.pivot_table(
    index='회원번호',
    columns='ISBN',
    values='value'
).fillna(0)

In [0]:
df_user_book_ratings.head()

ISBN,9780060741693,9780198480648,9780439113212,9780679864417,9780679884491,9780743461511,9780746054116,9780746054925,9781405052306,9781409505488,9781409520696,9781783442904,9788901028194,9788901033518,9788901052922,9788901054926,9788901072531,9788901101330,9788901114941,9788901114965,9788901121390,9788901162294,9788911021406,9788911027101,9788911030088,9788914017420,9788915073098,9788915084858,9788917198799,9788925548968,9788925550916,9788925551531,9788925554167,9788925554761,9788926387061,9788926387153,9788926390214,9788926391051,9788926391068,9788926391235,...,9788995151204,9788995501443,9788995688991,9788996134619,9788996134664,9788996145622,9788996640479,9788996746416,9788996842262,9788996881421,9788996991342,9788997089345,9788997174485,9788998515065,9791155427095,9791156390480,9791156394747,9791158461102,9791158461409,9791158510619,9791158680657,9791160486735,9791161231686,9791170260752,9791185205403,9791185343457,9791185343693,9791185431550,9791185431611,9791185431864,9791186340691,9791186472767,9791186560112,9791186560167,9791186602096,9791187481324,9791195049981,9791195568758,9791195788583,9788993632583
회원번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
687.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
829.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
matrix = df_user_book_ratings.values
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
# # matrix는 pivot_table 값을 numpy matrix로 만든 것 
# matrix = df_user_movie_ratings.as_matrix()

# user_ratings_mean은 사용자의 평균 평점 
user_rent_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_rent_mean.reshape(-1, 1)

In [0]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
matrix.shape

(500, 500)

In [0]:
user_rent_mean.shape

(500,)

In [0]:
matrix_user_mean.shape

(500, 500)

In [0]:
pd.DataFrame(matrix_user_mean, columns = df_user_book_ratings.columns).head()

ISBN,9780060741693,9780198480648,9780439113212,9780679864417,9780679884491,9780743461511,9780746054116,9780746054925,9781405052306,9781409505488,9781409520696,9781783442904,9788901028194,9788901033518,9788901052922,9788901054926,9788901072531,9788901101330,9788901114941,9788901114965,9788901121390,9788901162294,9788911021406,9788911027101,9788911030088,9788914017420,9788915073098,9788915084858,9788917198799,9788925548968,9788925550916,9788925551531,9788925554167,9788925554761,9788926387061,9788926387153,9788926390214,9788926391051,9788926391068,9788926391235,...,9788995151204,9788995501443,9788995688991,9788996134619,9788996134664,9788996145622,9788996640479,9788996746416,9788996842262,9788996881421,9788996991342,9788997089345,9788997174485,9788998515065,9791155427095,9791156390480,9791156394747,9791158461102,9791158461409,9791158510619,9791158680657,9791160486735,9791161231686,9791170260752,9791185205403,9791185343457,9791185343693,9791185431550,9791185431611,9791185431864,9791186340691,9791186472767,9791186560112,9791186560167,9791186602096,9791187481324,9791195049981,9791195568758,9791195788583,9788993632583
0,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006
1,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006
2,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,0.992,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,...,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
3,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006,-0.006
4,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,0.996,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,...,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004,-0.004


In [0]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [0]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(500, 12)
(12,)
(12, 500)


In [0]:
sigma = np.diag(sigma)

In [0]:
sigma.shape

(12, 12)

In [0]:
sigma[0]

array([3.64357473, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [0]:
sigma[1]

array([0.        , 3.65665544, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [0]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_rent_mean.reshape(-1, 1)

In [0]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_book_ratings.columns)
df_svd_preds.head()

ISBN,9780060741693,9780198480648,9780439113212,9780679864417,9780679884491,9780743461511,9780746054116,9780746054925,9781405052306,9781409505488,9781409520696,9781783442904,9788901028194,9788901033518,9788901052922,9788901054926,9788901072531,9788901101330,9788901114941,9788901114965,9788901121390,9788901162294,9788911021406,9788911027101,9788911030088,9788914017420,9788915073098,9788915084858,9788917198799,9788925548968,9788925550916,9788925551531,9788925554167,9788925554761,9788926387061,9788926387153,9788926390214,9788926391051,9788926391068,9788926391235,...,9788995151204,9788995501443,9788995688991,9788996134619,9788996134664,9788996145622,9788996640479,9788996746416,9788996842262,9788996881421,9788996991342,9788997089345,9788997174485,9788998515065,9791155427095,9791156390480,9791156394747,9791158461102,9791158461409,9791158510619,9791158680657,9791160486735,9791161231686,9791170260752,9791185205403,9791185343457,9791185343693,9791185431550,9791185431611,9791185431864,9791186340691,9791186472767,9791186560112,9791186560167,9791186602096,9791187481324,9791195049981,9791195568758,9791195788583,9788993632583
0,0.022607,-0.005131,0.017538,0.002261,0.015214,-0.012822,0.009341,-0.009635,0.003984,0.002367,-0.000210,0.000826,-0.000906,0.006284,0.007415,0.013221,0.012426,0.002404,0.005179,-0.002721,0.019436,-0.002120,0.002392,0.007393,-0.000765,-0.005385,-0.008019,0.028613,-0.001208,0.001159,0.012185,0.005101,0.000073,0.017249,0.052086,0.009410,-0.008502,0.025129,0.011191,0.015099,...,0.012838,-0.008618,0.005736,0.002635,0.007505,0.006259,-0.010345,0.015682,0.007217,0.006195,0.006408,0.011047,0.056389,0.009493,0.022508,-0.001559,0.035338,0.015139,0.004006,0.006692,0.001998,-0.006293,-0.003974,0.006301,0.012135,0.004106,-0.010533,0.003540,0.005753,-0.004653,0.003476,-0.002091,0.025474,-0.004727,0.004567,0.004996,0.010249,-0.022939,0.004164,0.005587
1,-0.003821,-0.006474,-0.003498,-0.002240,-0.021070,0.008311,-0.001937,0.006992,0.012249,-0.015594,-0.000048,0.022841,-0.004698,0.006476,0.005672,0.002252,-0.002982,-0.011103,0.002983,0.003816,0.001452,0.013201,0.020411,0.003063,0.023044,-0.009017,0.043035,0.067045,-0.011322,0.001689,0.007594,0.000880,0.009780,-0.003759,0.018716,0.002218,0.013318,0.003555,-0.004308,0.004188,...,0.003572,0.035778,-0.001433,0.011970,-0.006996,0.008525,0.006643,0.002789,0.002364,-0.002242,0.000555,0.006965,0.018776,0.008204,-0.039537,-0.043137,0.003020,0.000577,-0.003131,0.001841,0.010566,-0.004852,0.007506,0.006422,0.006507,-0.004083,-0.001677,0.000941,0.012708,0.011725,-0.000520,-0.007179,-0.003433,0.011017,-0.001923,0.012347,0.010970,0.021982,0.000338,0.008168
2,-0.013552,0.028518,0.003188,0.027834,0.002618,0.003407,-0.010027,0.000064,-0.000869,-0.001161,0.012132,0.035765,0.018327,-0.017712,0.003658,0.011126,0.005744,0.052935,0.030803,0.004000,-0.001561,0.000618,-0.005376,0.003539,0.012843,0.002268,0.020240,0.012984,-0.004671,0.001843,0.016360,0.010615,-0.009618,-0.014077,0.038698,0.000599,0.024655,-0.000926,0.010897,0.011069,...,0.010019,-0.033239,-0.008602,0.001565,0.130832,-0.009531,0.014328,-0.012396,0.003314,0.025660,0.008884,0.008514,0.012863,0.025732,0.011914,-0.007594,-0.001089,0.012598,0.012441,0.008116,0.012287,0.056291,-0.009557,0.009906,0.004040,0.002523,-0.014035,-0.025396,0.014637,0.003490,-0.002817,0.016993,-0.011386,0.011680,-0.005301,-0.000111,0.001693,0.117510,-0.001252,0.003399
3,-0.001632,0.000642,0.000527,0.022601,0.000556,0.012048,0.003244,-0.000030,0.018020,-0.003280,0.001150,-0.002569,-0.002167,0.012414,0.007267,0.008673,0.001892,-0.000317,0.001958,0.008809,-0.005592,0.011887,0.005320,0.003620,0.007209,0.010158,-0.007057,0.002176,-0.014148,0.005114,0.006552,0.001212,0.010946,-0.002111,0.001989,0.006326,0.006181,0.004774,-0.007246,0.006453,...,0.020511,0.043461,0.015544,0.015577,-0.016696,0.008776,0.007153,0.012762,0.011645,0.001055,0.001903,0.012506,0.005296,0.010525,-0.001091,-0.003700,-0.000597,0.022925,-0.000009,0.022078,0.008639,-0.001901,0.020512,-0.002602,0.015062,0.011266,

In [0]:
df_svd_preds.shape

(500, 500)

In [0]:
df_rentlist[df_rentlist.회원번호 == 134843]

,회원번호,ISBN,value
465,134843.0,9788934937487,1
466,134843.0,9788930231558,1
467,134843.0,9791186560112,1
468,134843.0,9788963720746,1
469,134843.0,9788930231367,1
470,134843.0,9788901072531,1


In [0]:
def recommend_movies(df_svd_preds, user_id, ori_book_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    # 우리꺼는 아이디가 뒤죽박죽이다 df_svd_preds 인덱스와 df_user_book_ratings 인덱스의 위치가 같도록 바꿔줘야함.
    # ex df_svd_preds의 index 0 은 126번의 책에 대한 예측값이다 (바꿔주기 꽤 어려워 보인다..)
    # 리스트에 인덱스 번호를 자동으로 추가해주는 enumerate 함수로 구했음!!
    # a는 인덱스인 회원번호를 정수형으로 변환
    # b는 enumerate에 조건문을 걸어 입력된 회원번호가 몇번째 인덱스인지 구하기
    # 값을 user_row_number에 저장
    userid_indexing = df_user_book_ratings.index.astype('int')
    userid_index = [i for i,j in enumerate(userid_indexing) if j == user_id]
    user_row_number = userid_index[0]
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.회원번호 == user_id]
    
    # 사용자 히스토리 불필요해서 삭제
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    # user_history = user_data.merge(ori_book_df, on = 'ISBN').sort_values(['value'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_book_df[~ori_book_df['ISBN'].isin(user_data['ISBN'])]

    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ISBN')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_data, recommendations

In [0]:
import time
start = time.time()  # 시작 시간 저장

already_rated, predictions = recommend_movies(df_svd_preds, 136640, df_books, df_rentlist, 10)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 0.014408588409423828


In [0]:
predictions

,Unnamed: 0,ISBN,도서명,저자,주제분류번호,Predictions
365,368,9788971996676,담론,신영복 지음,199.2,0.144449
272,275,9788957363058,(만화로 보는)그리스 로마 신화,토머스 불핀치 지음;이광진 엮음;서영 그림,219,0.125362
79,80,9788930201407,Why? 삼국유사,원전엮음: 홍영주 ;글구성: 박세준 ;그림: 송회석,082 911.03,0.100808
306,309,9788960242982,(Maplestory) 메이플 홈런왕,글·그림: 현보아트스쿨,695,0.096496
293,296,9788958286158,꿈꾸는 징검돌,김용철 쓰고 그림,653,0.085913
296,299,9788958621799,(어린이) 살아있는 과학 교과서,"홍준의,최후남,고현덕,김태일 원작 ;김태일 글 ;마정원 그림",400,0.081176
159,162,9788937810770,무인도에서 살아남기,최덕희 글;강경효 그림,400,0.075267
358,361,9788966070671,테일즈런너 수학킹왕짱!,글: 이강안 ;그림: 문인호,410,0.074116
355,358,9788966070350,테일즈런너 역사 킹왕짱!,글: 디지털터치 ;그림: 최익규,909,0.063250
104,105,9788930206723,Why? 과학수사,글: 조영선 ;만화: 이영호,517,0.060410


In [0]:
already_rated.head(10)

,회원번호,ISBN,value
1431,136640.0,9788930232524,1
1432,136640.0,9788926390214,1
1433,136640.0,9788932909974,1


In [0]:
predictions

,Unnamed: 0,ISBN,도서명,저자,주제분류번호,Predictions
365,368,9788971996676,담론,신영복 지음,199.2,0.144449
272,275,9788957363058,(만화로 보는)그리스 로마 신화,토머스 불핀치 지음;이광진 엮음;서영 그림,219,0.125362
79,80,9788930201407,Why? 삼국유사,원전엮음: 홍영주 ;글구성: 박세준 ;그림: 송회석,082 911.03,0.100808
306,309,9788960242982,(Maplestory) 메이플 홈런왕,글·그림: 현보아트스쿨,695,0.096496
293,296,9788958286158,꿈꾸는 징검돌,김용철 쓰고 그림,653,0.085913
296,299,9788958621799,(어린이) 살아있는 과학 교과서,"홍준의,최후남,고현덕,김태일 원작 ;김태일 글 ;마정원 그림",400,0.081176
159,162,9788937810770,무인도에서 살아남기,최덕희 글;강경효 그림,400,0.075267
358,361,9788966070671,테일즈런너 수학킹왕짱!,글: 이강안 ;그림: 문인호,410,0.074116
355,358,9788966070350,테일즈런너 역사 킹왕짱!,글: 디지털터치 ;그림: 최익규,909,0.063250
104,105,9788930206723,Why? 과학수사,글: 조영선 ;만화: 이영호,517,0.060410
